In [1]:
import numpy as np 
import pandas as pd # move to polaris
import os
from tqdm import tqdm
from time import sleep
import asyncio
import chess

In [28]:
!kaggle datasets download -d datasnaek/chess

chess.zip: Skipping, found more recently modified local copy (use --force to force download)


In [29]:
!unzip -qf chess.zip

In [2]:
games = pd.read_csv("games.csv")

In [32]:
async def seperateMoves1(game, moves):
    previousMovesList = []
    previousMoves = []
    targets = []
    numMoves = len(moves)
    for j in range(numMoves):
        previousMovesList.append(previousMoves.copy())
        previousMoves.append(moves[j])
        targets.append(moves[j])
    multipleGamesNoMoves = pd.concat([pd.DataFrame(game)]*numMoves, ignore_index = True) #might get rid of pd.dataframe
    return previousMovesList, targets, multipleGamesNoMoves

In [34]:
movesFrame = games.moves
gamesNoMoves = games.drop("moves", axis=1)
numGames = len(gamesNoMoves)
seperatedNoMoves = pd.DataFrame(columns=gamesNoMoves.columns)
seperatedGamesFutures = []
for i in tqdm(range(numGames)):
    seperatedGamesFutures.append(seperateMoves1(gamesNoMoves.iloc[[i]], movesFrame[i].split(" ")))

previousMovesListList = []
targetsList = []
multipleGamesNoMovesList = []
for i in tqdm(range(numGames)):
    previousMovesList, targets, multipleGamesNoMoves = (await asyncio.gather(seperatedGamesFutures[i]))[0]
    previousMovesListList.append(previousMovesList)
    targetsList.append(targets)
    multipleGamesNoMovesList.append(multipleGamesNoMoves)
previousMovesList = sum(previousMovesListList, [])
targets = sum(targetsList, [])
multipleGamesNoMoves = pd.concat(multipleGamesNoMovesList, ignore_index=True)

/tmp/ipykernel_5232/1646654467.py:5: RuntimeWarning: coroutine 'seperateMoves1' was never awaited
  seperatedGamesFutures = []
100%|██████████| 20058/20058 [27:00<00:00, 12.38it/s]  


In [35]:
clean = multipleGamesNoMoves.loc[:,["white_rating", "black_rating"]]
clean["target"] = targets
clean["previousMoves"] = previousMovesList

In [36]:
clean.to_csv("clean.csv")

In [3]:
clean = pd.read_csv("clean.csv")

In [4]:
clean.head()

,Unnamed: 0,white_rating,black_rating,target,previousMoves
0,0,1500,1191,d4,[]
1,1,1500,1191,d5,['d4']
2,2,1500,1191,c4,"['d4', 'd5']"
3,3,1500,1191,c6,"['d4', 'd5', 'c4']"
4,4,1500,1191,cxd5,"['d4', 'd5', 'c4', 'c6']"


In [5]:
clean.iloc[range(14),:]

,Unnamed: 0,white_rating,black_rating,target,previousMoves
0,0,1500,1191,d4,[]
1,1,1500,1191,d5,['d4']
2,2,1500,1191,c4,"['d4', 'd5']"
3,3,1500,1191,c6,"['d4', 'd5', 'c4']"
4,4,1500,1191,cxd5,"['d4', 'd5', 'c4', 'c6']"
5,5,1500,1191,e6,"['d4', 'd5', 'c4', 'c6', 'cxd5']"
6,6,1500,1191,dxe6,"['d4', 'd5', 'c4', 'c6', 'cxd5', 'e6']"
7,7,1500,1191,fxe6,"['d4', 'd5', 'c4', 'c6', 'cxd5', 'e6', 'dxe6']"
8,8,1500,1191,Nf3,"['d4', 'd5', 'c4', 'c6', 'cxd5', 'e6', 'dxe6',..."
9,9,1500,1191,Bb4+,"['d4', 'd5', 'c4', 'c6', 'cxd5', 'e6', 'dxe6',..."


In [31]:
lengthOfGames = []
length = 0
for moves in clean.loc[:,"previousMoves"]:
    if moves == '[]':
        lengthOfGames.append(length+1)
        length = 0
    else:
        length += 1
lengthOfGames.append(length+1)
lengthOfGames = lengthOfGames[1:]

In [30]:
startingIndexOfGame = 0
fens = []
for length in tqdm(lengthOfGames):
    board = chess.Board()
    moves = clean.iloc[startingIndexOfGame:startingIndexOfGame + length]["target"]
    for SANmove in moves:
        PGNmove = board.parse_san(SANmove)
        board.push(PGNmove)
        fens.append(board.fen())
    startingIndexOfGame += length

100%|██████████| 20058/20058 [07:10<00:00, 46.61it/s] 


In [32]:
clean["fen"] = fens

In [33]:
clean.to_csv("cleanWithFen.csv")

In [34]:
cleanWithFen = pd.read_csv("cleanWithFen.csv")

In [35]:
cleanWithFen.head()

,Unnamed: 0.1,Unnamed: 0,white_rating,black_rating,target,previousMoves,fen
0,0,0,1500,1191,d4,[],rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR ...
1,1,1,1500,1191,d5,['d4'],rnbqkbnr/ppp1pppp/8/3p4/3P4/8/PPP1PPPP/RNBQKBN...
2,2,2,1500,1191,c4,"['d4', 'd5']",rnbqkbnr/ppp1pppp/8/3p4/2PP4/8/PP2PPPP/RNBQKBN...
3,3,3,1500,1191,c6,"['d4', 'd5', 'c4']",rnbqkbnr/pp2pppp/2p5/3p4/2PP4/8/PP2PPPP/RNBQKB...
4,4,4,1500,1191,cxd5,"['d4', 'd5', 'c4', 'c6']",rnbqkbnr/pp2pppp/2p5/3P4/3P4/8/PP2PPPP/RNBQKBN...
